# G2Config Reference

**!! If you wish to run any of Senzing's Java notebooks navigate to "Help" -> "Launch Classic Notebook" !!**

## Prepare Environment

In [ ]:
import com.senzing.g2.engine.G2Config;
import com.senzing.g2.engine.G2ConfigJNI;
import com.senzing.g2.engine.Result;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
String str;
public static void RenderJSON(Object obj){
    str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

String senzingLicenseBase64 = System.getenv("SENZING_LICENSE_BASE64_ENCODED");

// Construct the JSON string used for Senzing Engine configuration.
String senzingConfigJson;
if(senzingLicenseBase64!=null){
    senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"LICENSESTRINGBASE64\": \"" + senzingLicenseBase64 + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
}

else{
    senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
   
}
RenderJSON(senzingConfigJson);

## G2Config

The G2Config API is used to create a new JSON configuration from the default template
or manipulate a JSON configuration that it is provided.
The API read/writes a JSON string, but that string would typically be
read/written from/to a g2config.json file that the G2 engine reads during the initV2(...) call.

### Initialize G2Config

###### Parameters

- **moduleName:** (str) A short name given to this instance of the G2Config
  object.
- **senzingConfigJson:** (str) A JSON string containing configuration parameters.
- **verboseLogging:** (bool) A boolean which enables diagnostic logging.

In [ ]:
String moduleName = "ExampleG2Config";
boolean verboseLogging = true;

G2Config g2config = new G2ConfigJNI();
int return_code = g2config.init(moduleName, senzingConfigJson, verboseLogging);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### Create Configuration handle
`create()` creates an in-memory handle for an editable configuration document.  It does so by pulling from the default template config document, stored in the product installation.

In [ ]:
Result<Long> config = new Result<Long>();
long configHandle;
int return_code = g2config.create(config);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else{
    configHandle = config.getValue();
    System.out.print(config.getValue());
}

### save
`save()` takes an in-memory handle for an editable configuration document, and exports it as a string through the function parameters.

###### Parameters

- **configHandle:** (int) variable containg the the number of the used config
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.save(configHandle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    RenderJSON(response);

### load
`load()` creates an in-memory handle for an editable configuration document. It does so by loading the config document, provided as an input parameter string.

###### Parameters

- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **config:** (long) the handle of the config object to get the config json data from

In [ ]:
// example using the saved config from the `save()` method
g2config.load(response.toString(), config);

## Datasources

### listDataSources

`listDataSources()` returns a list of data sources contained in an in-memory handle for an editable configuration document

###### Parameters

- **configHandle:** (long) the handle of the config object to get the config json data from
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listDataSources(configHandle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    RenderJSON(response);

### addDataSource

`addDataSource()` adds a new data source to an in-memory handle for an editable configuration document

###### Parameters

- **configHandle:** (long) the handle of the config object to get the config json data from
- **dataSource:** (str) json containing the name of the datasource to be added
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
String dataSource = "{\"DSRC_CODE\": \"CUSTOMER\"}";

StringBuffer response = new StringBuffer();

int return_code = g2config.addDataSource(configHandle, dataSource, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    RenderJSON(response);

### listDataSources again

Call G2Config's `listDataSources()` method and pretty-print results. Notice that the list now contains the newly added datasource_code of "CUSTOMER".

Parameters
- **configHandle:** (long) the handle of the config object to get the config json data from
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listDataSources(configHandle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    RenderJSON(response);

### deleteDataSource

`deleteDataSource()` removes a data source from an in-memory handle for an editable configuration document

###### Parameters

- **configHandle:** (long) the handle of the config object to get the config json data from
- **dataSource:** (str) json containing the name of the datasource to be added

In [ ]:
String dataSource = "{\"DSRC_CODE\": \"COMPANY\"}";

int return_code = g2config.deleteDataSource(configHandle, dataSource);

if(return_code!=0)
    System.out.print(g2config.getLastException());

## Cleanup

### close
`close()` deletes an in-memory handle for an editable configuration document, and frees it’s internal memory resources.

**Parameters**
- **configHandle:** (long) the handle of the config object to get the config json data from

In [ ]:
g2config.close(configHandle);

### destroy

In [ ]:
int return_code = g2config.destroy();

if(return_code!=0)
    System.out.print(g2config.getLastException());